In [1]:
struct Variable
	name::Symbol
	r::Int # number of possible values
end

const Assignment = Dict{Symbol,Int}
const FactorTable = Dict{Assignment,Float64}

struct Factor
	vars::Vector{Variable}
	table::FactorTable
end
variablenames(ϕ::Factor) = [var.name for var in ϕ.vars]

select(a::Assignment, varnames::Vector{Symbol}) =Assignment(n=>a[n] for n in varnames)

function assignments(vars::AbstractVector{Variable})
    names = [var.name for var in vars]
    return vec([Assignment(n=>v for (n,v) in zip(names, values))
    			for values in product((1:v.r for v in vars)...)])
end

function normalize!(ϕ::Factor)
	z = sum(p for (a,p) in ϕ.table)
	for (a,p) in ϕ.table
		ϕ.table[a] = p/z
	end
	return ϕ
end


normalize! (generic function with 1 method)

In [4]:
#= struct MyVariable
    myName::Symbol
    rr::Int
end =#
qq_variablenames(tall::Int) = [tall * 2 for i in range(0,0, tall)]
qq_variablenames(10)

10-element Vector{Int64}:
 20
 20
 20
 20
 20
 20
 20
 20
 20
 20

In [6]:
Base.Dict{Symbol,V}(a::NamedTuple) where {V} =
    Dict{Symbol,V}(n => v for (n, v) in zip(keys(a), values(a)))

Base.convert(::Type{Dict{Symbol,V}}, a::NamedTuple) where {V} =
    Dict{Symbol,V}(a)

Base.isequal(a::Dict{Symbol,<:Any}, nt::NamedTuple) =
    length(a) == length(nt) &&
    all(a[n] == v for (n, v) in zip(keys(nt), values(nt)))


In [7]:
X = Variable(:x, 2)
Y = Variable(:y, 2)
Z = Variable(:z, 2)
PHI = Factor([X, Y, Z], FactorTable(
(x=1, y=1, z=1) => 0.08, (x=1, y=1, z=2) => 0.31,
(x=1, y=2, z=1) => 0.09, (x=1, y=2, z=2) => 0.37,
(x=2, y=1, z=1) => 0.01, (x=2, y=1, z=2) => 0.05,
(x=2, y=2, z=1) => 0.02, (x=2, y=2, z=2) => 0.07,
))


Factor(Variable[Variable(:x, 2), Variable(:y, 2), Variable(:z, 2)], Dict(Dict(:y => 2, :z => 2, :x => 1) => 0.37, Dict(:y => 1, :z => 2, :x => 1) => 0.31, Dict(:y => 2, :z => 1, :x => 2) => 0.02, Dict(:y => 1, :z => 1, :x => 1) => 0.08, Dict(:y => 1, :z => 2, :x => 2) => 0.05, Dict(:y => 2, :z => 2, :x => 2) => 0.07, Dict(:y => 2, :z => 1, :x => 1) => 0.09, Dict(:y => 1, :z => 1, :x => 2) => 0.01))

In [ ]:
struct BayesianNetwork
    vars::Vector{Variable}
    factors::Vector{Factor}
    graph::SimpleDiGraph{Int64}
end

In [ ]:
function probability(bn::BayesianNetwork, assignment)
    subassignment(ϕ) = select(assignment, variablenames(ϕ))
    probability(ϕ) = get(ϕ.table, subassignment(ϕ), 0.0)
    return prod(probability(ϕ) for ϕ in bn.factors)
end


In [18]:
a = Dict{Symbol,Integer}((a=1, b=2, c=3))

Dict{Symbol, Integer} with 3 entries:
  :a => 1
  :b => 2
  :c => 3

In [15]:
a[:a]

1

In [16]:
a["a"]

KeyError: KeyError: key "a" not found

In [11]:
:a

:a

In [13]:
:marius

:marius

In [6]:
struct SetCategorical{S}
    elements::Vector{S} # Set elements (could be repeated)
    distr::Categorical # Categorical distribution over set elements
    function SetCategorical(elements::AbstractVector{S}) where {S}
        weights = ones(length(elements))
        return new{S}(elements, Categorical(normalize(weights, 1)))
    end
    function SetCategorical(
        elements::AbstractVector{S},
        weights::AbstractVector{Float64}
    ) where {S}
        ℓ₁ = norm(weights, 1)
        if ℓ₁ < 1e-6 || isinf(ℓ₁)
            return SetCategorical(elements)
        end
        distr = Categorical(normalize(weights, 1))
        return new{S}(elements, distr)
    end
end

Distributions.rand(D::SetCategorical) = D.elements[rand(D.distr)]

Distributions.rand(D::SetCategorical, n::Int) = D.elements[rand(D.distr, n)]

function Distributions.pdf(D::SetCategorical, x)
    sum(e == x ? w : 0.0 for (e, w) in zip(D.elements, D.distr.p))
end



UndefVarError: UndefVarError: Distributions not defined